In [29]:
from PIL import Image
import numpy as np
import pickle

In [30]:
im = Image.open('./nosync.nosync/slope_md_res10.tif')
slopearray = np.array(im)
print(slopearray.shape)

(1409, 1179)


In [50]:
import rioxarray

rds = rioxarray.open_rasterio('./nosync.nosync/slope_md_res10.tif')
df = rds.to_dataframe(name="slope")

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import shapefile

polygons = []
sf = shapefile.Reader("Reference data/slope_deformations_reference_polygons/reference_data_existing_landslides_polygons.shp")
shapes = sf.shapes()
rectangles = []

for shape in shapes:
    polygons.append(Polygon(shape.points))
    rectangles.append(shape.bbox)

is_landslide = set()
for i in range(len(polygons)):
    rectangle = rectangles[i]
    low_x, low_y, high_x, high_y = rectangle[0], rectangle[1], rectangle[2], rectangle[3]
    small_df = df.query('y >= @low_y and y <= @high_y and x >= @low_x and x <= @high_x')
    if(len(small_df) > 0):
        for _, y, x in small_df.index:
            if polygons[i].contains(Point(x, y)):
                is_landslide.add((x, y))

bools = np.array([1 if (x, y) in is_landslide else 0 for _, y, x in df.index])
df.insert(loc = 0, column='is_in', value=bools)
rds = df.reset_index().drop(columns=['band', 'spatial_ref'])

In [56]:
def preprocess_numpy_array(coordinates, original):
    return coordinates['is_in'].to_numpy().reshape(original.shape[0], original.shape[1])

In [57]:
X_test = list()
Y_test = list()

size_of_square = 100
overlap = 30

try:
    with open('mdtest.pickle', 'rb') as handle:
        X_test, Y_test = pickle.load(handle)
except:
    np_coords: np.ndarray = preprocess_numpy_array(rds, slopearray)
    for i in range(0, slopearray.shape[0] - size_of_square,overlap):
        for j in range(0, slopearray.shape[1] - size_of_square, overlap):
            current_array: np.ndarray = slopearray[i:i+size_of_square, j:j+size_of_square].flatten()
            X_test.append(current_array)
            Y_test.append(np.mean(np_coords[i:i+size_of_square, j:j+size_of_square]))
    
    with open('mdtest.pickle', 'wb') as handle:
        pickle.dump((X_test, Y_test), handle)

In [58]:
from sklearn.linear_model import LinearRegression

x = np.array(X_test)
y = np.array(Y_test)

model = LinearRegression().fit(x, y)

In [59]:
def get_coordinates(i, j, coordinates, original):
    top_left = coordinates.loc[i * original.shape[1] + j]
    top_right = coordinates.loc[i * original.shape[1] + j + size_of_square]
    bottom_left = coordinates.loc[(i+size_of_square) * original.shape[1] + j]
    bottom_right = coordinates.loc[(i + size_of_square) * original.shape[1] + j + size_of_square]
    return [top_left, top_right, bottom_left, bottom_right]

In [60]:
import math
whole_slopes = np.array(Image.open('./nosync.nosync/slope_whole_res10.tif'))
result = []

rds2 = rioxarray.open_rasterio('./nosync.nosync/slope_whole_res10.tif')
rds2 = rds2.to_dataframe(name="slope").reset_index().drop(columns=['band', 'spatial_ref', 'slope'])

for i in range(0, whole_slopes.shape[0] - size_of_square,overlap):
    for j in range(0, whole_slopes.shape[1] - size_of_square, overlap):
        current_array: np.ndarray = whole_slopes[i:i+size_of_square, j:j+size_of_square].flatten()
        coordinates = get_coordinates(i, j, rds2, whole_slopes)
        coordinates.append(model.predict(np.array([current_array])))
        result.append(coordinates)

result = np.array(result, dtype=object)